Landslide Susceptibility	dis.007	https://earthobservatory.nasa.gov/IOTD/view.php?id=89937

Global Landslide Susceptibility Map dis.007 https://pmm.nasa.gov/applications/global-landslide-model

There are 2 summary maps (2007, 2013?)
* Static
* GFW and SRTM data helped improve from 2007 -> 2013

POTENTIALLY ADD NEW DATASET: Also - landslide catalog, stored in RW_Data/Rasters as well

Import libraries

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3 tools

In [2]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/dis_007_landslide_susceptibility_map/"

s3_file = "dis_007_landslide_susceptibility_map.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [3]:
local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/"
file_name = "global_landscape_susceptibility_2013_static.tif"

local_orig = local_folder + file_name

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "edit.tif" 

Use rasterio to reproject and compress

In [4]:
with rio.open(local_orig, 'r') as src:
    print(src.profile)
    data = src.read()
    
print(data)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 127.0, 'width': 43200, 'height': 15841, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.00833333333333333, 0.0, 72.00006000333326, 0.0, -0.00833333333333333), 'affine': Affine(0.00833333333333333, 0.0, -180.0,
       0.0, -0.00833333333333333, 72.00006000333326), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'interleave': 'band'}
[[[  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  ..., 
  [127 127 127 ..., 127 127 127]
  [127 127 127 ..., 127 127 127]
  [127 127 127 ..., 127 127 127]]]


In [6]:
# Note - this is the core of Vizz's netcdf2tif function

with rio.open(local_orig, 'r') as src:
    # This assumes data is readable by rasterio
    # May need to open instead with netcdf4.Dataset, for example
   
    data = src.read()[0]
    
    rows = data.shape[0]
    columns = data.shape[1]
    
    print(rows)
    print(columns)
    
    # Latitude bounds
    south_lat = -90
    north_lat = 90

    # Longitude bounds
    west_lon = -180
    east_lon = 180
    
    transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    
    # Profile
    
    no_data_val = 127
    target_projection = 'EPSG:4326'
    target_data_type = np.uint8
    
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':target_data_type, 
        'crs':target_projection, 
        'transform':transform, 
        'compress':'lzw', 
        'nodata': no_data_val
    }
    
    with rio.open(local_edit, "w", **profile) as dst:
        dst.write(data.astype(profile["dtype"]), 1)

15841
43200


Upload orig and edit files to s3

In [7]:
# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))

/Users/nathansuberi/Desktop/RW_Data/Rasters/global_landscape_susceptibility_2013_staticedit.tif  40936406 / 40936406.0  (100.00%)